In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions

In [4]:
# re-size all the images
IMAGE_SIZE = [224, 224]
train_path = '../Car Brand/Train'
valid_path = '../Car Brand/Test'

In [ ]:
# import resnet50 Library and add preprocessing layer to the front of resnet50
# we will use imagenet weights

resnet = ResNet50(
    input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False
)

In [ ]:
resnet.summary()

In [ ]:
# don't train existing layers
for layer in resnet.layers:
    layer.trainable = False # don't train images

In [6]:
# useful for getting number of output classes
folders = glob('../Car Brand/Train/*')
folders

['../Car Brand/Train\\audi',
 '../Car Brand/Train\\lamborghini',
 '../Car Brand/Train\\mercedes']

In [ ]:
# layers
x = keras.layers.Flatten()(resnet.output)

In [ ]:
# use dense to set the length of folders as output
prediction = keras.layers.Dense(len(folders), activation='softmax')(x)

# create a model object
model = keras.models.Model(inputs=resnet.input, output=prediction)

In [ ]:
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# use the image Data Generator to import/read the images from the dataset/folder
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
) # in test data don't perform data augmentation

In [ ]:
# make sure you provide the same target size as initialized for the image size

training_set = train_datagen.flow_from_directory(
    '../Car Brand/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
test_set = test_datagen.flow_from_directory(
    '../Car Brand/Test',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
# fit the model
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs=50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set)
)

In [ ]:
r.history

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train_loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# plot the loss
plt.plot(r.history['accuracy'], label='train_acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
model.save('model_resnet50.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
img_data = load_model('model_resnet50.h5')

In [ ]:
img_data

In [ ]:
img = image.load_img('../Car Brand/Test/lamborghini/10.jpg', target_size=(224, 224))

In [ ]:
x = image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a = np.argmax(model.predict(img_data), axis=1)

In [ ]:
a==1